In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [2]:
ruta_archivo1 = "Datasets/Light Duty Vehicles.csv"
df = pd.read_csv(ruta_archivo1)

print(df.head())

   Vehicle ID  Fuel ID  Fuel Configuration ID  Manufacturer ID  Category ID  \
0       13044       45                    9.0              365           27   
1       12854       45                    9.0              377           27   
2       12842       45                    9.0              377           27   
3       12783       45                    9.0              377           27   
4       12782       45                    9.0              377           27   

                Model  Model Year  Alternative Fuel Economy City  \
0                 NSX        2022                            NaN   
1                 A3         2022                            NaN   
2          A3 quattro        2022                            NaN   
3  A4 allroad quattro        2022                            NaN   
4          A4 quattro        2022                            NaN   

   Alternative Fuel Economy Highway  Alternative Fuel Economy Combined  ...  \
0                               NaN  

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3008 entries, 0 to 3007
Data columns (total 29 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Vehicle ID                          3008 non-null   int64  
 1   Fuel ID                             3008 non-null   int64  
 2   Fuel Configuration ID               2973 non-null   float64
 3   Manufacturer ID                     3008 non-null   int64  
 4   Category ID                         3008 non-null   int64  
 5   Model                               3008 non-null   object 
 6   Model Year                          3008 non-null   int64  
 7   Alternative Fuel Economy City       1595 non-null   float64
 8   Alternative Fuel Economy Highway    1505 non-null   float64
 9   Alternative Fuel Economy Combined   467 non-null    float64
 10  Conventional Fuel Economy City      2044 non-null   float64
 11  Conventional Fuel Economy Highway   2046 no

## I Eliminar columnas que no vamos a utilizar

In [4]:
df.drop(columns=[ 'Notes', 'Drivetrain', 'PHEV Total Range', 'Engine Cylinder Count','Transmission Type', 'Engine Description','Fuel Configuration ID', 'Manufacturer URL', 'Fuel Code', 'PHEV Type' ], axis=1, inplace=True)

## II Exploración y tratamiento de Nulos

### 1 Columna Category

In [5]:
# Filtramos el dataset para incluir solo "Sedans" y "SUVs"
df = df[df['Category'].isin(['Sedan', 'SUV'])]



### 2 Columnas Alternative Fuel Economy City, Alternative Fuel Economy Highway y Alternative Fuel Economy Combined

Reemplazamos los nulos eficiencia cominada con el promedio entre eficiencia en ciudad y eficiencia en carretera.

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 778 entries, 17 to 2946
Data columns (total 19 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Vehicle ID                          778 non-null    int64  
 1   Fuel ID                             778 non-null    int64  
 2   Manufacturer ID                     778 non-null    int64  
 3   Category ID                         778 non-null    int64  
 4   Model                               778 non-null    object 
 5   Model Year                          778 non-null    int64  
 6   Alternative Fuel Economy City       456 non-null    float64
 7   Alternative Fuel Economy Highway    437 non-null    float64
 8   Alternative Fuel Economy Combined   155 non-null    float64
 9   Conventional Fuel Economy City      602 non-null    float64
 10  Conventional Fuel Economy Highway   603 non-null    float64
 11  Conventional Fuel Economy Combined  256 non-null

In [7]:
# Calcular el promedio entre 'Alternative Fuel Economy City' y 'Alternative Fuel Economy Highway'
df['Average City Highway'] = (df['Alternative Fuel Economy City'] + df['Alternative Fuel Economy Highway']) / 2

# Reemplazar los valores nulos en 'Alternative Fuel Economy Combined' con el promedio calculado
df['Alternative Fuel Economy Combined'] = df['Alternative Fuel Economy Combined'].fillna(df['Average City Highway'])

# Eliminar la columna auxiliar 'Average City Highway'
df.drop('Average City Highway', axis=1, inplace=True)

### 3 Columnas Conventional Fuel Economy City, Conventional Fuel Economy Highway y Alternative Fuel Economy Combined

Reemplazamos los nulos eficiencia cominada con el promedio entre eficiencia en ciudad y eficiencia en carretera.

In [8]:
# Calcular el promedio entre 'Alternative Fuel Economy City' y 'Alternative Fuel Economy Highway'
df['Average City Highway'] = (df['Conventional Fuel Economy City'] + df['Conventional Fuel Economy Highway']) / 2

# Reemplazar los valores nulos en 'Alternative Fuel Economy Combined' con el promedio calculado
df['Conventional Fuel Economy Combined'] = df['Alternative Fuel Economy Combined'].fillna(df['Average City Highway'])

# Eliminar la columna auxiliar 'Average City Highway'
df.drop('Average City Highway', axis=1, inplace=True)

### 4 Columna Engine Size

Modificamos el nombre de los datos para hacerlos más comprensibles

In [9]:
df['Engine Size'].unique()

array(['141 kW and 172 kW electric motor', '138kW electric motor',
       '157kW electric motor', '80 kW and 150 kW electric motor', '2.0L',
       '3.0L', '335 kW electric motor', '190 and 230 kW electric motors',
       '2.5L', '3.3L', '3.3L V6', '198 kW electric motors (x2)',
       '258 kW electric motors (x2)', '179  kW electric motors (X2)',
       '179  kW electric motor', '198 kW electric motor',
       '216 kW electric motor', '74 kW and 165 kW electric motors',
       '125 kW electric motor', '168 kW electric motor',
       '150 kW electric motor', '120 kW electric motor', '1.6L', '5.7L',
       '3.6L', '3.5L', '4.0L', '2.4L', '3.0L ', '162, 162, 163, 163 kW ',
       '243 and 248 kW electric motors', '230kW electric motors (X3)',
       '91 and 200 kW electric motors', '133 and 179 kW electric motors',
       '209 kW electric motor', '80 and 150 kW electric motors',
       '150 kW electric motors (X2)', '150kW electric motor', nan,
       '180 and 189 kW electric motor', '15

In [10]:
# Usar expresiones regulares para extraer el número antes de 'kW' o 'kw' y luego agregar ' kW'
df['Engine Size'] = df['Engine Size'].str.replace(r'(\d+)\s*[kK][wW].*', r'\1 kW', regex=True)

# Mostrar los valores únicos resultantes
print(df['Engine Size'].unique())


['141 kW' '138 kW' '157 kW' '80 kW' '2.0L' '3.0L' '335 kW'
 '190 and 230 kW' '2.5L' '3.3L' '3.3L V6' '198 kW' '258 kW' '179 kW'
 '216 kW' '74 kW' '125 kW' '168 kW' '150 kW' '120 kW' '1.6L' '5.7L' '3.6L'
 '3.5L' '4.0L' '2.4L' '3.0L ' '162, 162, 163, 163 kW' '243 and 248 kW'
 '230 kW' '91 and 200 kW' '133 and 179 kW' '209 kW' '80 and 150 kW' nan
 '180 and 189 kW' '151 and 273 kW' '69 and 201 kW' '5.3L' '201 kW' '2.0L '
 '193 and 205 kW' '205 and 375 kW' '158 and 203 kW' '147 and 211 kW'
 '158 and 219 kW' '95 kW' '147 kW' '2.0' '193 kW' '193, 205, 375 kW'
 '5.3L ' '3.5L ' '3.7L' '5.6L' '193/193 kW' '193/375 kW' '2.4' '3.6' '5.3'
 '5.4' '3.5' '3.7' '100' '2.5' '3.0' '5.0' '2.04' '5.6' '5.7' '6.2L'
 '5.4L' '5.0L' '115' '2.0 L' '6.0L' '115 kW' '4.6L' '1.8L' '4.4L' '2.2L'
 '4.3L' '4.7L' '5.3 L' '6.0 L' '4.7 L' '2.3 L' '5.6 L' '2.4 L' '3.3 L'
 '4.7' '2.3L' '4.0 Liter']


In [11]:
# Definimos una función para transformar los valores de la columna según las reglas proporcionadas
def transform_engine_size(value):
    if pd.isna(value):
        return value  # Retornamos el valor NaN tal como está
    if 'Liter' in value:  # Si el valor termina en 'Liter', reemplazamos 'Liter' por 'L'
        return value.replace('Liter', 'L')
    if value[-1].isdigit():  # Si el último carácter es un dígito, agregamos 'L' al final
        return f'{value}L'
    return value  # Retornamos el valor sin cambios si no cumple las condiciones anteriores

# Aplicamos la función a la columna 'Engine Size' del dataset filtrado
df['Engine Size'] = df['Engine Size'].apply(transform_engine_size)

# Verificamos algunos valores únicos después de la transformación para confirmar los cambios
df['Engine Size'].unique()


array(['141 kW', '138 kW', '157 kW', '80 kW', '2.0L', '3.0L', '335 kW',
       '190 and 230 kW', '2.5L', '3.3L', '3.3L V6L', '198 kW', '258 kW',
       '179 kW', '216 kW', '74 kW', '125 kW', '168 kW', '150 kW',
       '120 kW', '1.6L', '5.7L', '3.6L', '3.5L', '4.0L', '2.4L', '3.0L ',
       '162, 162, 163, 163 kW', '243 and 248 kW', '230 kW',
       '91 and 200 kW', '133 and 179 kW', '209 kW', '80 and 150 kW', nan,
       '180 and 189 kW', '151 and 273 kW', '69 and 201 kW', '5.3L',
       '201 kW', '2.0L ', '193 and 205 kW', '205 and 375 kW',
       '158 and 203 kW', '147 and 211 kW', '158 and 219 kW', '95 kW',
       '147 kW', '193 kW', '193, 205, 375 kW', '5.3L ', '3.5L ', '3.7L',
       '5.6L', '193/193 kW', '193/375 kW', '5.4L', '100L', '5.0L',
       '2.04L', '6.2L', '115L', '2.0 L', '6.0L', '115 kW', '4.6L', '1.8L',
       '4.4L', '2.2L', '4.3L', '4.7L', '5.3 L', '6.0 L', '4.7 L', '2.3 L',
       '5.6 L', '2.4 L', '3.3 L', '2.3L', '4.0 L'], dtype=object)

### 5 Columna Engine Size

Ordenamos en categorìas los datos para que puedan ser más manejables.

In [12]:
df['Engine Type'].unique()

array(['e-motor', 'SI', 'CI', 'FC', nan, 'Diesel', 'V6, Hybrid',
       'GM Vortec engine', 'Vortec 5300', 'ECOTEC', 'SOHC', 'Hybrid',
       '5-speed automatic', 'DOHC 32-valve', 'DOHC 24-valve with V V T-i',
       '6.0 L Vortec V8', '4.7 L SOHC V8', 'Duratec I-4 Atkinson Cycle',
       'Not Available', '2.4 L ECOTEC', 'DOHC 24-valve with VVT-i',
       'Not available', 'V8',
       '(DOHC) 24 valve with (VVT-i)  and  Electric permanent magne',
       '(DOHC) 24 valve with (VVT-i)', 'Permanent-Magnet 50 kW DC'],
      dtype=object)

In [13]:
# Definimos una función para clasificar los valores de 'Engine Type' en las categorías proporcionadas
def classify_engine_type(value):
    # Categorías según descripción
    si_keywords = ["Gasoline", "Petrol", "SI"]
    e_motor_keywords = ["Electric", "e-motor"]
    ci_keywords = ["Diesel", "CI"]
    fc_keywords = ["Fuel Cell", "Hydrogen", "FC"]
    
    # Clasificación
    if pd.isna(value):
        return value  # Mantenemos NaN sin cambios
    value_lower = value.lower()  # Convertimos a minúsculas para una comparación insensible a mayúsculas
    if any(keyword.lower() in value_lower for keyword in si_keywords):
        return "SI"
    elif any(keyword.lower() in value_lower for keyword in e_motor_keywords):
        return "e-motor"
    elif any(keyword.lower() in value_lower for keyword in ci_keywords):
        return "CI"
    elif any(keyword.lower() in value_lower for keyword in fc_keywords):
        return "FC"
    else:
        return "Other"  # Para valores que no coinciden con ninguna categoría

# Aplicamos la función de clasificación a la columna 'Engine Type'
df['Engine Type'] = df['Engine Type'].apply(classify_engine_type)

# Verificamos las categorías únicas resultantes
df['Engine Type'].unique()


array(['e-motor', 'SI', 'CI', 'FC', nan, 'Other'], dtype=object)

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 778 entries, 17 to 2946
Data columns (total 19 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Vehicle ID                          778 non-null    int64  
 1   Fuel ID                             778 non-null    int64  
 2   Manufacturer ID                     778 non-null    int64  
 3   Category ID                         778 non-null    int64  
 4   Model                               778 non-null    object 
 5   Model Year                          778 non-null    int64  
 6   Alternative Fuel Economy City       456 non-null    float64
 7   Alternative Fuel Economy Highway    437 non-null    float64
 8   Alternative Fuel Economy Combined   489 non-null    float64
 9   Conventional Fuel Economy City      602 non-null    float64
 10  Conventional Fuel Economy Highway   603 non-null    float64
 11  Conventional Fuel Economy Combined  741 non-null

## III Outliers y duplicados

- Outliers en Vehicle ID y Manufacturer ID: Los ID de vehículo y fabricante parecen ser identificadores numéricos. Los outliers aquí podrían indicar vehículos o fabricantes que son significativamente diferentes en términos de su identificación numérica, lo que podría no ser relevante desde una perspectiva analítica.

- Outliers en Model Year:Los años de modelo que son considerablemente más antiguos que el resto podrían indicar vehículos que son significativamente diferentes en términos de tecnología, diseño, eficiencia de combustible, etc.

- Outliers en Economía de Combustible Alternativo y Convencional (City, Highway, Combined):Los valores extremadamente altos o bajos en la economía de combustible (tanto para combustibles alternativos como convencionales) pueden indicar vehículos que son excepcionalmente eficientes o ineficientes. 

In [15]:
# Seleccionamos solo las columnas numéricas
df_numeric = df.select_dtypes(include=['float64', 'int64'])

# Calcula el rango intercuartílico (IQR) para cada columna numérica
Q1 = df_numeric.quantile(0.25)
Q3 = df_numeric.quantile(0.75)
IQR = Q3 - Q1

# Identifica los outliers
outliers = ((df_numeric < (Q1 - 1.5 * IQR)) | (df_numeric > (Q3 + 1.5 * IQR)))

# Muestra los outliers
for column in outliers.columns:
    if outliers[column].any():
        print(f"Outliers in {column}:")
        print(df_numeric[column][outliers[column]])
        print()


Outliers in Vehicle ID:
2378    968
2379    946
2382    960
2388    980
2389    966
       ... 
2878     82
2920    582
2921     50
2945    596
2946    595
Name: Vehicle ID, Length: 128, dtype: int64

Outliers in Manufacturer ID:
17      377
18      377
19      377
20      377
21      377
       ... 
1951    391
1960    383
1979    377
1980    377
2241    391
Name: Manufacturer ID, Length: 138, dtype: int64

Outliers in Model Year:
2945    1998
2946    1998
Name: Model Year, dtype: int64

Outliers in Alternative Fuel Economy City:
17       78.0
20       76.0
23      100.0
24      100.0
33      100.0
        ...  
1411     89.0
1571     90.0
1572     89.0
1969     78.0
2129     78.0
Name: Alternative Fuel Economy City, Length: 79, dtype: float64

Outliers in Alternative Fuel Economy Highway:
17      77.0
18      75.0
19      64.0
20      78.0
21      78.0
        ... 
1411    90.0
1571    94.0
1572    90.0
1969    74.0
2129    74.0
Name: Alternative Fuel Economy Highway, Length: 91, dty

In [16]:
# Eliminamos outlier
df['Conventional Fuel Economy Highway'] = df['Conventional Fuel Economy Highway'].replace(231.0, pd.NA)



No hay duplicados

In [17]:
# Buscamos duplicados en todo el DataFrame
duplicates = df.duplicated()

# Contamos el número de duplicados
number_of_duplicates = duplicates.sum()

# Muestra el número de duplicados
print(f'Número de filas duplicadas: {number_of_duplicates}')

# SVemos las filas duplicadas:
duplicated_rows = df[duplicates]
print(duplicated_rows)

Número de filas duplicadas: 0
Empty DataFrame
Columns: [Vehicle ID, Fuel ID, Manufacturer ID, Category ID, Model, Model Year, Alternative Fuel Economy City, Alternative Fuel Economy Highway, Alternative Fuel Economy Combined, Conventional Fuel Economy City, Conventional Fuel Economy Highway, Conventional Fuel Economy Combined, Engine Type, Engine Size, Manufacturer, Category, Fuel, Fuel Configuration Name, Electric-Only Range]
Index: []


In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 778 entries, 17 to 2946
Data columns (total 19 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Vehicle ID                          778 non-null    int64  
 1   Fuel ID                             778 non-null    int64  
 2   Manufacturer ID                     778 non-null    int64  
 3   Category ID                         778 non-null    int64  
 4   Model                               778 non-null    object 
 5   Model Year                          778 non-null    int64  
 6   Alternative Fuel Economy City       456 non-null    float64
 7   Alternative Fuel Economy Highway    437 non-null    float64
 8   Alternative Fuel Economy Combined   489 non-null    float64
 9   Conventional Fuel Economy City      602 non-null    float64
 10  Conventional Fuel Economy Highway   602 non-null    object 
 11  Conventional Fuel Economy Combined  741 non-null

### IV Guardar dataframe

In [19]:
# Restablecer el índice del DataFrame
df.reset_index(drop=True, inplace=True)

In [20]:
# Guardar el DataFrame en un archivo CSV
df.to_csv('Clean_Light_Duty.csv', index=False)